In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/CV/yolov7

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/11trdxIFS-4ZCbq6mNHOeVwUfdN9adKKo/CV/yolov7


In [ ]:
# Gather all Q3 images
!cp -Rf ./data/CityCam/Q3/170/* ./data/CityCam/Q3_all/
!cp -Rf ./data/CityCam/Q3/173/* ./data/CityCam/Q3_all/
!cp -Rf ./data/CityCam/Q3/398/* ./data/CityCam/Q3_all/
!cp -Rf ./data/CityCam/Q3/410/* ./data/CityCam/Q3_all/
!cp -Rf ./data/CityCam/Q3/495/* ./data/CityCam/Q3_all/
!cp -Rf ./data/CityCam/Q3/511/* ./data/CityCam/Q3_all/

In [ ]:
import os
import glob
Q3_image_paths = glob.glob(os.path.join('./data/CityCam/Q3_all', '*.jpg'))
print(len(Q3_image_paths))

1200


In [ ]:
!python detect.py --weights "./runs/train/Q2/bus/weights/best.pt" --source "./data/CityCam/Q3_all" --save-txt --nosave --classes 2 5 7 --exist-ok --no-trace

Namespace(agnostic_nms=False, augment=False, classes=[2, 5, 7], conf_thres=0.25, device='', exist_ok=True, img_size=640, iou_thres=0.45, name='exp', no_trace=True, nosave=True, project='runs/detect', save_conf=False, save_txt=True, source='./data/CityCam/Q3_all', update=False, view_img=False, weights=['./runs/train/Q2/bus/weights/best.pt'])
YOLOR 🚀 2022-8-2 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6194944 gradients
/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Done. (110.560s)


In [ ]:
Q3_final_paths = []
for i in Q3_image_paths:
    path = "runs/detect/exp/labels/" + i.split('.jpg')[-2].split(os.sep)[-1] + ".txt"
    if os.path.exists(path):
      Q3_final_paths.append(i)

In [ ]:
import random
ques = 'Q2'

CLASS_NAME = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
]

# Q2 and Q3 TODO : select "better" images from Q2 folder
def select_imaegs(image_paths, images_num=200):
    """
    :param image_paths: --> ['your_folder/images1.jpg', 'your_folder/images2.jpg', ...]
    :param images_num: choose the number of images
    :return :
        selected_image_paths = ['your_folder/images10.jpg', 'your_folder/images12.jpg', ...]
    """
    # TODO : select images
    # return random.sample(image_paths, images_num)
    print(len(image_paths))
    image_labels = {}
    selected_image_paths = []
    for j in image_paths:
        if j.split(os.sep)[-2] not in image_labels:
          image_labels[j.split(os.sep)[-2]] = {}
        f = open(j.split('.jpg')[-2]+'.txt', 'r')
        label = f.read().split()
        count = 0
        for i in range(int(len(label)/5)):
            if float(label[i*5 + 3]) * float(label[i*5 + 4]) > 0.004:
                count += 1
            if label[i*5] == '5':
                count += 10
        image_labels[j.split(os.sep)[-2]][j] = count
    for i in image_labels.keys():
      selected_image_paths += sorted(image_labels[i])[:int(images_num/len(image_labels.keys()))]
    print(len(selected_image_paths))

    return selected_image_paths

# TODO : split train and val images
def split_train_val_path(all_image_paths, train_val_ratio=0.9, is_Q3 = False):
    """
    :param all_image_paths: all image paths for question in the data folder
    :param train_val_ratio: ratio of image paths used to split training and validation
    :return :
        train_image_paths = ['your_folder/images1.jpg', 'your_folder/images2.jpg', ...]
        val_image_paths = ['your_folder/images3.jpg', 'your_folder/images4.jpg', ...]
    """
    # TODO : split train and val
    # train_image_paths = all_image_paths[: int(len(all_image_paths) * train_val_ratio)]  # just an example
    # val_image_paths = all_image_paths[int(len(all_image_paths) * train_val_ratio):]  # just an example
    folders = {}
    if is_Q3:
        for i in all_image_paths:
            if i.split(os.sep)[-1][:3] not in folders:
                folders[i.split(os.sep)[-1][:3]] = []
            folders[i.split(os.sep)[-1][:3]].append(i)
        train_image_paths = []
        val_image_paths = []
        for i in folders.keys():
            length = len(folders[i])
            train_image_paths += folders[i][:int(length * train_val_ratio)]
            val_image_paths += folders[i][int(length * train_val_ratio) : length]
        return train_image_paths, val_image_paths

    for i in all_image_paths:
        if i.split(os.sep)[-2] not in folders:
            folders[i.split(os.sep)[-2]] = []
        folders[i.split(os.sep)[-2]].append(i)
    train_image_paths = []
    val_image_paths = []
    for i in folders.keys():
        length = len(folders[i])
        train_image_paths += folders[i][:int(length * train_val_ratio)]
        val_image_paths += folders[i][int(length * train_val_ratio) : length]

    # print(len(train_image_paths))

    return train_image_paths, val_image_paths

# Get whole and Test image paths
all_image_paths = glob.glob(os.path.join('./data/CityCam', ques, '*', '*.jpg'))
test_image_paths = glob.glob(os.path.join('./data/CityCam', 'test', '*' + os.sep + '*.jpg'))

# for Q2 and Q3 : select images
if ques == 'Q2' or ques == 'Q3':
    selected_image_paths = select_imaegs(all_image_paths, images_num=200)
else:
    selected_image_paths = all_image_paths
# split Train and Val
train_image_paths, val_image_paths = split_train_val_path(selected_image_paths)
print(len(train_image_paths))
print(len(val_image_paths))
train_Q3_paths, val_Q3_paths = split_train_val_path(Q3_image_paths, is_Q3 = True)
train_image_paths += train_Q3_paths
val_image_paths += val_Q3_paths
print(len(train_image_paths))
print(len(val_image_paths))
# write train/val/test info
train_path = os.path.join('./data/CityCam', 'train.txt')
val_path = os.path.join('./data/CityCam', 'val.txt')
test_path = os.path.join('./data/CityCam', 'test.txt')
with open(train_path, 'w') as f:
    for image_path in train_image_paths:
        f.write(os.path.abspath(image_path) + '\n')
with open(val_path, 'w') as f:
    for image_path in val_image_paths:
        f.write(os.path.abspath(image_path) + '\n')
with open(test_path, 'w') as f:
    for image_path in test_image_paths:
        f.write(os.path.abspath(image_path) + '\n')

# write training YAML file
with open('./data/citycam.yaml', 'w') as f:
    f.write("train: " + os.path.abspath(train_path) + "\n")
    f.write("val: " + os.path.abspath(val_path) + "\n")
    f.write("test: " + os.path.abspath(test_path) + "\n")
    # number of classes
    f.write('nc: 80\n')
    # class names
    f.write('names: ' + str(CLASS_NAME))

# delete cache
if os.path.exists(os.path.join('./data/CityCam', 'train.cache')):
    os.remove(os.path.join('./data/CityCam', 'train.cache'))
if os.path.exists(os.path.join('./data/CityCam', 'val.cache')):
    os.remove(os.path.join('./data/CityCam', 'val.cache'))

1200
198
174
24
1254
144


In [ ]:
!cp -Rf ./runs/detect/exp/labels/* ./data/CityCam/Q3_all/

In [ ]:
# training command
!python train.py --project runs/train/Q3 --name all

YOLOR 🚀 2022-8-2 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=12, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='./data/citycam.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='all', noautoanchor=False, nosave=False, notest=False, project='runs/train/Q3', quad=False, rect=False, resume=False, save_dir='runs/train/Q3/all', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=12, upload_dataset=False, weights='yolov7.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train/Q3', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0,

In [ ]:
# testing command
!python test.py --verbose --show_div_cams --task test --weights "./runs/train/Q3/all/weights/best.pt" --project runs/test/Q3 --name all

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/citycam.yaml', device='', exist_ok=False, img_size=640, iou_thres=0.65, name='all', no_trace=False, project='runs/test/Q3', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, show_div_cams=True, single_cls=False, task='test', verbose=True, weights=['./runs/train/Q3/all/weights/best.pt'])
YOLOR 🚀 2022-8-2 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined

In [ ]:
# training command
!python train.py --weights "./runs/train/Q2/bus/weights/best.pt" --project runs/train/Q3 --name all_pre

YOLOR 🚀 2022-8-2 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=12, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='./data/citycam.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='all_pre', noautoanchor=False, nosave=False, notest=False, project='runs/train/Q3', quad=False, rect=False, resume=False, save_dir='runs/train/Q3/all_pre2', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=12, upload_dataset=False, weights='./runs/train/Q3/all/weights/best.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train/Q3', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=

In [ ]:
# testing command
!python test.py --verbose --show_div_cams --task test --weights "./runs/train/Q3/all_pre/weights/best.pt" --project runs/test/Q3 --name all_pre

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/citycam.yaml', device='', exist_ok=False, img_size=640, iou_thres=0.65, name='all_pre2', no_trace=False, project='runs/test/Q3', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, show_div_cams=True, single_cls=False, task='test', verbose=True, weights=['./runs/train/Q3/all_pre2/weights/best.pt'])
YOLOR 🚀 2022-8-2 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[at

In [ ]:
# training command
!python train.py --weights "./runs/train/Q2/bus/weights/best.pt" --project runs/train/Q3 --name pre_freeze --freeze 2

YOLOR 🚀 2022-8-2 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=12, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='./data/citycam.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, freeze=[2], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='pre_freeze', noautoanchor=False, nosave=False, notest=False, project='runs/train/Q3', quad=False, rect=False, resume=False, save_dir='runs/train/Q3/pre_freeze2', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=12, upload_dataset=False, weights='./runs/train/Q3/all_pre2/weights/best.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train/Q3', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warm

In [ ]:
# testing command
!python test.py --verbose --show_div_cams --task test --weights "./runs/train/Q3/pre_freeze/weights/best.pt" --project runs/test/Q3 --name pre_freeze

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/citycam.yaml', device='', exist_ok=False, img_size=640, iou_thres=0.65, name='pre_freeze2', no_trace=False, project='runs/test/Q3', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, show_div_cams=True, single_cls=False, task='test', verbose=True, weights=['./runs/train/Q3/pre_freeze2/weights/best.pt'])
YOLOR 🚀 2022-8-2 torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36905341 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ign